# 10 - Linear MHD equations

Topics covered in this tutorial:

- instance of [ShearAlfven](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.ShearAlfven) propagator
- instance of [Magnetosonic](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.Magnetosonic) propagator
- initialization with noise
- power spectrum plot
- $\theta$-pinch and $Z$-pinch configurations

We are concerned with the solution of the ideal, linearized MHD equations coded in the model [LinearMHD](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_fluid.html#struphy.models.fluid.LinearMHD):

$$
\begin{align}
 &\frac{\partial \tilde \rho}{\partial t}+\nabla\cdot(\rho_0 \tilde{\mathbf{U}})=0\,, 
        \\[2mm]
        \rho_0&\frac{\partial \tilde{\mathbf{U}}}{\partial t} + \nabla \tilde p
        = (\nabla \times \tilde{\mathbf{B}})\times \mathbf{B}_0 + (\nabla\times\mathbf{B}_0)\times \tilde{\mathbf{B}} \,,
        \\[2mm]
        &\frac{\partial \tilde p}{\partial t} + \nabla\cdot(p_0 \tilde{\mathbf{U}}) 
        + \frac{2}{3}\,p_0\nabla\cdot \tilde{\mathbf{U}}=0\,,
        \\[2mm]
        &\frac{\partial \tilde{\mathbf{B}}}{\partial t} - \nabla\times(\tilde{\mathbf{U}} \times \mathbf{B}_0)
        = 0\,.
\end{align}
$$

## MHD dispersion relation in a slab

In [ ]:
# set up domain Omega
from struphy.geometry.domains import Cuboid

xL = 0.0
xR = 1.0
yL = 0.0
yR = 1.0
zL = 0.0
zR = 60.0
domain = Cuboid(l1=xL, r1=xR, l2=yL, r2=yR, l3=zL, r3=zR)

In [ ]:
# set up MHD equilibrium
from struphy.fields_background.equils import HomogenSlab

B0x = 0.0
B0y = 1.0
B0z = 1.0
beta = 1.0
n0 = 1.0
mhd_equil = HomogenSlab(B0x=B0x, B0y=B0y, B0z=B0z, beta=beta, n0=n0)

# must set domain of Cartesian MHD equilibirum
mhd_equil.domain = domain

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [1, 1, 64]
p = [1, 1, 3]
spl_kind = [True, True, True]
derham = Derham(Nel, p, spl_kind)

In [ ]:
# create solution field u in Vh_2 subset H(div)
u_space = "Hdiv"  # choose 'H1vec' for comparison
mhd_u = derham.create_spline_function("velocity", u_space)

# create solution field B in Vh_2 subset H(div)
b_field = derham.create_spline_function("magnetic field", "Hdiv")

# create solution fields rho and p in Vh_3 subset L2
mhd_rho = derham.create_spline_function("mass density", "L2")
mhd_p = derham.create_spline_function("pressure", "L2")

In [ ]:
# initial perturbations
pert_params_u = {
    "noise": {
        "comps": [True, True, True],
        "direction": "e3",
        "amp": 0.1,
        "seed": None,
    }
}

In [ ]:
mhd_u.initialize_coeffs(pert_params=pert_params_u)
b_field.initialize_coeffs()
mhd_rho.initialize_coeffs()
mhd_p.initialize_coeffs()

In [ ]:
# evalaute at logical coordinates
import numpy as np

e1 = 0.5
e2 = 0.5
e3 = np.linspace(0, 1, 100)

u_vals = mhd_u(e1, e2, e3, squeeze_out=True)
b_vals = b_field(e1, e2, e3, squeeze_out=True)

In [ ]:
# plot inital conditions

from matplotlib import pyplot as plt

plt.figure(figsize=(10, 6))
for i in range(3):
    plt.subplot(2, 3, i + 1)
    plt.plot(e3, u_vals[i])
    plt.title(f"$\hat u^{2 if u_space == 'Hdiv2' else ' '}_{i + 1}$")
    plt.xlabel("$\eta_3$")
    if i == 0:
        plt.ylabel("a.u.")

    plt.subplot(2, 3, i + 4)
    plt.plot(e3, b_vals[i])
    plt.title(f"$\hat b^2_{i + 1}$")
    plt.xlabel("$\eta_3$")
    if i == 0:
        plt.ylabel("a.u.")

In [ ]:
# set up mass matrices
from struphy.feec.mass import WeightedMassOperators

mass_ops = WeightedMassOperators(derham, domain, eq_mhd=mhd_equil)

In [ ]:
# set up basis projection operators
from struphy.feec.basis_projection_ops import BasisProjectionOperators

basis_ops = BasisProjectionOperators(derham, domain, eq_mhd=mhd_equil)

In [ ]:
# pass simulation parameters to Propagators
from struphy.propagators.base import Propagator

Propagator.derham = derham
Propagator.domain = domain
Propagator.mass_ops = mass_ops
Propagator.basis_ops = basis_ops

In [ ]:
from struphy.propagators.propagators_fields import Magnetosonic, ShearAlfven

# default parameters of Propagator
opts = ShearAlfven.options(default=True)
opts

In [ ]:
# default parameters of Propagator
opts = Magnetosonic.options(default=True)
opts

In [ ]:
prop_1 = ShearAlfven(mhd_u.vector, b_field.vector, u_space=u_space)
prop_1_explicit = ShearAlfven(mhd_u.vector, b_field.vector, u_space=u_space, algo="rk4")

In [ ]:
prop_2 = Magnetosonic(mhd_rho.vector, mhd_u.vector, mhd_p.vector, u_space=u_space, b=b_field.vector)

In [ ]:
# time stepping, with both propagators
Tend = 180.0 - 1e-6
dt = 0.15

u_of_t = {}
p_of_t = {}
time = 0.0
n = 0
while time < Tend:
    n += 1

    # advance in time
    prop_1(dt)
    prop_2(dt)
    time += dt

    # evaluate solution
    u_of_t[time] = mhd_u(e1, e2, e3)
    p_of_t[time] = [mhd_p(e1, e2, e3)]

    if n % 100 == 0:
        print(f"{n}/{int(np.ceil(Tend / dt))} steps completed.")

In [ ]:
# reset initial condition
mhd_u.initialize_coeffs(pert_params=pert_params_u)
b_field.initialize_coeffs()

In [ ]:
# time stepping, with both propagators
Tend = 180.0 - 1e-6
dt = 0.15

u_of_t_ex = {}
p_of_t_ex = {}
time = 0.0
n = 0
while time < Tend:
    n += 1

    # advance in time
    prop_1_explicit(dt)
    prop_2(dt)
    time += dt

    # evaluate solution
    u_of_t_ex[time] = mhd_u(e1, e2, e3)
    p_of_t_ex[time] = [mhd_p(e1, e2, e3)]

    if n % 100 == 0:
        print(f"{n}/{int(np.ceil(Tend / dt))} steps completed.")

In [ ]:
from struphy.diagnostics.diagn_tools import power_spectrum_2d

x, y, z = domain(e1, e2, e3)

# equilibrium pressure
p0 = beta * (B0x**2 + B0y**2 + B0z**2) / 2

disp_params = {"B0x": B0x, "B0y": B0y, "B0z": B0z, "p0": p0, "n0": n0, "gamma": 5 / 3}

# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(
    u_of_t,
    "mhd_u",
    "notebook tutorial",
    grids=[e1, e2, e3],
    grids_mapped=[x, y, z],
    component=0,
    slice_at=[0, 0, None],
    do_plot=True,
    disp_name="MHDhomogenSlab",
    disp_params=disp_params,
)

In [ ]:
from struphy.diagnostics.diagn_tools import power_spectrum_2d

x, y, z = domain(e1, e2, e3)

# equilibrium pressure
p0 = beta * (B0x**2 + B0y**2 + B0z**2) / 2

disp_params = {"B0x": B0x, "B0y": B0y, "B0z": B0z, "p0": p0, "n0": n0, "gamma": 5 / 3}

# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(
    u_of_t_ex,
    "mhd_u",
    "notebook tutorial",
    grids=[e1, e2, e3],
    grids_mapped=[x, y, z],
    component=0,
    slice_at=[0, 0, None],
    do_plot=True,
    disp_name="MHDhomogenSlab",
    disp_params=disp_params,
)

In [ ]:
power_spectrum_2d(
    p_of_t,
    "mhd_p",
    "notebook tutorial",
    grids=[e1, e2, e3],
    grids_mapped=[x, y, z],
    component=0,
    slice_at=[0, 0, None],
    do_plot=True,
    disp_name="MHDhomogenSlab",
    disp_params=disp_params,
)

In [ ]:
power_spectrum_2d(
    p_of_t_ex,
    "mhd_p",
    "notebook tutorial",
    grids=[e1, e2, e3],
    grids_mapped=[x, y, z],
    component=0,
    slice_at=[0, 0, None],
    do_plot=True,
    disp_name="MHDhomogenSlab",
    disp_params=disp_params,
)

## $\theta$-pinch stability

Under construction ...

## $Z$-pinch stability

Under construction ...

## Screw-pinch modes

In [ ]:
# set up domain Omega
from struphy.geometry.domains import HollowCylinder

a = 1
R0 = 3

a1 = 0.0 + 1e-6
a2 = a
Lz = 2 * np.pi * R0
domain = HollowCylinder(a1=a1, a2=a2, Lz=Lz)

In [ ]:
# set up MHD equilibrium
from struphy.fields_background.equils import ScrewPinch

mhd_equil = ScrewPinch(a=a, R0=R0)

# must set domain of Cartesian MHD equilibirum
mhd_equil.domain = domain

In [ ]:
mhd_equil.plot_profiles()

In [ ]:
mhd_equil.show()

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [16, 32, 8]
p = [1, 1, 1]
spl_kind = [False, True, True]
derham = Derham(Nel, p, spl_kind)

# ..under construction

In [ ]:
# # initial perturbations
# pert_params = {}
# pert_params['type'] = 'ModesCos'

# noise_params = {
#             'comps' : {
#                 'velocity' : [True, True, True],
#             },
#             'direction' : 'e3',
#             'amp' : 0.1,
#             'seed' : None,
# }

# pert_params['noise'] = noise_params
# pert_params

In [ ]:
# # create solution field u in Vh_2 subset H(div)
# u_space = 'Hdiv' # choose 'H1vec' for comparison
# mhd_u = derham.create_spline_function('velocity', u_space, pert_params=pert_params)

# # create solution field B in Vh_2 subset H(div)
# b_field = derham.create_spline_function('magnetic field', 'Hdiv', pert_params=pert_params)

# # create solution fields rho and p in Vh_3 subset L2
# mhd_rho = derham.create_spline_function('mass density', 'L2', pert_params=pert_params)
# mhd_p = derham.create_spline_function('pressure', 'L2', pert_params=pert_params)

In [ ]:
# mhd_u.initialize_coeffs()
# b_field.initialize_coeffs()
# mhd_rho.initialize_coeffs()
# mhd_p.initialize_coeffs()

In [ ]:
# # evalaute at logical coordinates
# import numpy as np

# e1 = .5
# e2 = .5
# e3 = np.linspace(0, 1, 100)

# u_vals = mhd_u(e1, e2, e3, squeeze_out=True)
# b_vals = b_field(e1, e2, e3, squeeze_out=True)

In [ ]:
# # plot inital conditions

# from matplotlib import pyplot as plt

# plt.figure(figsize=(10, 6))
# for i in range(3):
#     plt.subplot(2, 3, i + 1)
#     plt.plot(e3, u_vals[i])
#     plt.title(f'$\hat u^{2 if u_space == "Hdiv2" else " "}_{i + 1}$')
#     plt.xlabel('$\eta_3$')
#     if i == 0:
#         plt.ylabel('a.u.')

#     plt.subplot(2, 3, i + 4)
#     plt.plot(e3, b_vals[i])
#     plt.title(f'$\hat b^2_{i + 1}$')
#     plt.xlabel('$\eta_3$')
#     if i == 0:
#         plt.ylabel('a.u.')